In [1]:
import nltk
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.test.utils import datapath
from gensim import utils
from sklearn.decomposition import PCA

nltk.download('punkt')

%matplotlib inline

[nltk_data] Downloading package punkt to /Users/psachdeva/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
path = '../data/full_data.xlsx'
data = pd.read_excel(path)

In [3]:
# Grab text columns
text_cols = data.filter(regex='^full_text')

In [4]:
# Merge text columns into a single column
data['text'] = text_cols.apply(
    lambda x: ' '.join(x.dropna()),
    axis=1)

In [5]:
# Lowercase
data['text'] = data['text'].str.lower()
# Apostrophes
data['text'] = data['text'].str.replace('[’+]', '', regex=True)
# Hyphens
data['text'] = data['text'].str.replace('[-]', '', regex=True)
# Replace punctuation with space
data['text'] = data['text'].str.replace('[^\w\s]', ' ', regex=True)
# Remove new lines
data['text'] = data['text'].str.replace('\n', ' ', regex=True)

In [6]:
# Convert British English spellings to American English
def americanize(string):
    url = "https://raw.githubusercontent.com/hyperreality/American-British-English-Translator/master/data/british_spellings.json"
    british_to_american_dict = requests.get(url).json()    

    for british_spelling, american_spelling in british_to_american_dict.items():
        string = string.replace(british_spelling, american_spelling)
  
    return string

In [7]:
data['text'] = data['text'].apply(lambda x: americanize(x))
data['text'] = data['text'].str.replace('diskrimination', 'discrimination') # Fixing an typo in the json file
data['text'] = data['text'].str.replace('diskourse', 'discourse') # Fixing an typo in the json file
data['text'] = data['text'].str.replace(' dis ', '')

In [8]:
sentences = data['text'].to_list()
sentences = [nltk.word_tokenize(s) for s in sentences]

In [9]:
min_count = 5
vector_size = 32

In [10]:
model = Word2Vec(sentences=sentences,
                 min_count=min_count,
                 vector_size=vector_size)

In [ ]:
model.wv.most_similar('race')